<a href="https://colab.research.google.com/github/MuhammadAbuSufian/predicting-image/blob/main/Clip_Predicting_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests
import torch

# Load CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Define the function to create an image grid
def image_grid(imgs, cols):
    rows = (len(imgs) + cols - 1) // cols
    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

# Define the URLs of images from the COCO dataset
image_urls = [
    'http://images.cocodataset.org/val2014/COCO_val2014_000000159977.jpg',
    'http://images.cocodataset.org/val2014/COCO_val2014_000000311295.jpg',
    'http://images.cocodataset.org/val2014/COCO_val2014_000000457834.jpg',
    'http://images.cocodataset.org/val2014/COCO_val2014_000000555472.jpg',
    'http://images.cocodataset.org/val2014/COCO_val2014_000000174070.jpg',
    'http://images.cocodataset.org/val2014/COCO_val2014_000000460929.jpg'
]

# Load images from URLs
images = []
for url in image_urls:
    images.append(Image.open(requests.get(url, stream=True).raw))

# Display images in a grid
grid = image_grid(images, cols=3)
display(grid)

# Perform image classification
classes = ['giraffe', 'zebra', 'elephant']
inputs = processor(text=classes, images=images, return_tensors="pt", padding=True, do_convert_rgb=False)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # This is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  # Calculate softmax probabilities

# Print the classification results
for i, prob in enumerate(probs):
    pred_class_idx = torch.argmax(prob)
    pred_class = classes[pred_class_idx]
    print(f"Image {i+1}: Predicted class is '{pred_class}' with probability {prob[pred_class_idx]:.2f}")